# CUP IT 2023 Data Science

## Постановка задачи

Предложите механизм сортировки комментариев к постам по их популярности на основе методов
машинного обучения, чтобы модель могла как можно лучше ранжировать пользовательские ком-
ментарии.

Для этого:
1. Проведите проверку и разведочный анализ данных (EDA) и подумайте, какие готовые решения можно использовать для представления текста.
2. Используя тренировочную и тестовую выборки датасета, обучите модель ранжировать текстовые комментарии в порядке их популярности (от популярных к менее популярным) и проведите валидацию своей модели. Выбор модели необходимо аргументировать, основываясь на результатах обучения.
3. Проанализируйте полученные результаты и сформулируйте полезные инсайты о том, что обычно содержит популярный комментарий, чтобы команда VK могла использовать эту информацию для улучшения комментариев своих пользователей.
4. Предложите методы взаимодействия с комментаторами, а также механизмы поддержки для разных групп пользователей, включая тех, чьи комментарии непопулярны.

# 1. EDA

In [42]:
import numpy as np
import pandas as pd

Прочитаем файлы

In [49]:
train = pd.read_json('ranking_train.jsonl',  lines = True)
train = pd.concat([train[['text']], pd.json_normalize(train.comments)], axis=1)
train = pd.melt(train, id_vars='text', value_vars=[0, 1, 2, 3, 4])
train.sort_values(by=['text'], inplace=True)
train = pd.concat([train[['text']], train[['value']].iloc[:,0].apply(pd.Series)], axis=1)

KeyboardInterrupt: 

In [48]:
train

,text,variable,value
132334,"\t Mac OS X 10.8 restricted to App Store, signed apps by default",1,"{'text': 'From the article:&#62; When Apple first unveiled the Mac App Store, many - including myself - were concerned what it would mean for the future of the general purpose computer. It felt like a first step towards losing control and ownership over our own computers, a dreaded future where everything we do on our machines is curated, tracked, and monitored by companies who want to squeeze ever more money from us, and governments who want to control us.&#62; We're allowing an entire generation to be raised with the idea that you do not own software, that you do not own hardware, that you are not allowed to tinker with the magic smiles machine. This is going to come back to bite us in the ass in the future, when we're going to be faced with a shortage of low-level, hardcore programmers.In many ways, Richard Stallman has been right all along, and slowly, many of his fears are coming true.', 'score': 1}"
396655,"\t Mac OS X 10.8 restricted to App Store, signed apps by default",4,"{'text': 'The title is incorrect: OS X 10.8 (Mountain Lion) by default can only run signed apps. All appstore apps are signed apps, as are anyone who gets a free signing cert from Apple.There is a setting to ""only install Appstore Apps"" but it is not the default.', 'score': 4}"
220441,"\t Mac OS X 10.8 restricted to App Store, signed apps by default",2,"{'text': 'Appalling but I can't really fault them for responding to overwhelming incentives. Apple already gave their customers a chance to be treated like tool-using human beings, and the result was a decade of barely moving enough boutique desktops to keep their lights on. Now they basically emboss ""suck it, consumer"" along the rounded corners, and they're swimming in money. This move is an indictment of the people who demonstrably want to be treated like this, so when they get screwed by stuff their vendor chose not to let them do, make sure they realize it's their fault for opting in. Unlike Microsoft, Apple never conspired to keep competitors out of the market.', 'score': 2}"
308548,"\t Mac OS X 10.8 restricted to App Store, signed apps by default",3,"{'text': 'What is it about the ""geek"" mindset that makes one jump to the conclusion that this is part of a vast conspiracy, some cabal of OS vendors, to lock out hackers and prevent anyone from modifying their own machine? Isn't the simpler explanation that 99% of Mac users don't know, don't care how to install custom apps or homebrew, and would gladly have Apple restrict them to signed-apps-only if it means a significantly reduced chance of virus/trojan infection?And you know what? If some day, for some reason, Apple does decide to lock out hackers and tinkerers from their systems, somehow I doubt Apple's stock-holders will lose any sleep over it. For all the kvetching that I frequently hear from the HN crowd at large about ""let the market decide"", it's surprising how quickly that attitude vanishes when ""the market"" decides against you.(For what it's worth, there was a time when developers had to lay out far more than the average user for capable systems. It's only relatively recently that standard consumer machines could stand in for a dev box. I wouldn't be surprised if the future has developers, once again, paying premium dolar for hardware.)', 'score': 3}"
44227,"\t Mac OS X 10.8 restricted to App Store, signed apps by default",0,"{'text': 'The headline is fairly misleading, and the article sloppily mixes actual fact with predictions stated as fact.The reality: 10.8 by default only runs apps which were either downloaded from the App Store, with the iOS-style sandboxing that will require, or signed with Apple's new GateKeeper service, and otherwise a traditional Mac app.Those who condemn this as a flagrant loss of freedom are lost in the philosophical weeds. The conventional wisdom before the iPhone was that native apps were all but dead a

In [38]:
df = pd.concat([df[['text']], pd.json_normalize(df.comments)], axis=1)


In [40]:
df.head()

,text,0,1,2,3,4
0,How many summer Y Combinator fundees decided not to continue with their startup and go back to school? and what were the reasons?,"{'text': 'Going back to school is not identical with giving up. Some founders go back to school and keep working on the startup while there. However, those do so much worse than the people who work on the startup full-time that going back to school seems, in practice, not too far removed from a death sentence for a startup.Off the top of my head, I'd guess we've had about 8 startups where the founders went back to school. It doesn't only happen with summer batches. Founders from winter batches do it too.Usually the reason is that the startup isn't doing very well. However, that judgement depends a lot on how determined the founders are. One reason we now shy away from funding people still in school is that they often unconsciously want the startup to fail, because the idea of dropping out frightens them.A lot of startups look bad after 3 months. Someone who's out of school and has to make it work or get a job in a cubicle will say ""don't worry, we'll figure out how to make it successful.""', 'score': 0}","{'text': 'There will invariably be those who don't see the success they set out for, and they fall back to their original path. That's why a founder's commitment is so critical. When your one plan is to become a startup founder, regardless of school or any other factors, you will do your damndest to make something of value and succeed. It's part _doing_ it better than the next guy and _wanting_ it more than the next guy.', 'score': 1}","{'text': 'For me school is a way to be connected to what is going on in the ""real world"". I entered school thinking it is EITHER school or entrepreneurship. But in last year my views have changed. I really think there is a middle path where you do JUST enough to stay in school all the while trying out new ventures. (May be the Steve Jobs route?)School is an awesome platform to launch your site off and get quick feedback - ESPECIALLY if your venture fits in with the social networking ecosystem.', 'score': 2}","{'text': 'I guess it really depends on how hungry you are and how much you believe in your product. I'm only 24 and still in school as of the moment, yet I'm more than willing to leave school in order to initiate my start up. Come to think of it, I'm willing to travel half the world just to get advice and connections from YC. ', 'score': 3}","{'text': 'I know pollground decided to go back to school after getting y combinator funding', 'score': 4}"
1,CBS acquires last.fm for $280m,"{'text': 'It will be curious to see where this heads in the long run. CBS is on a tear but will it fit their image, will they try and establish control, overall agenda. I've enjoyed last.fm for many years supporting through paypal donations each time I expire...it'll be interesting.', 'score': 0}","{'text': 'Does this mean that there's now a big-name company who will fight for the repeal of the recent streaming-music royalty hike?', 'score': 1}","{'text': 'Also on BBC News: http://news.bbc.co.uk/1/low/technology/6701863.stm .Nice to see a London-based co. hit the headlines.', 'score': 2}","{'text': 'I don't understand what they do that is worth $70M a year. ', 'score': 3}","{'text': 'sold out too cheaply. given their leadership position, they should have ask for at least $500m', 'score': 4}"
2,How Costco Became the Anti-Wal-Mart,"{'text': 'I really hate it when people falsely claim that the fiduciary responsibilities of a public company includes maximizing profits at every opportunity. Thinking long-term and win-win is the truly responsible CEOs job. Costco is the Google of discount warehouses. Wal-mart is Microsoft.', 'score': 0}","{'text': 'I love Costco. For me, the best quote of the article was this:""The traditional retailer will say: 'I'm selling this for $10. I wonder whether I can get $10.50 or $11.' We say: 'We're selling it for $9. How do we get it down

In [33]:
pd.options.display.max_colwidth = 5000

In [34]:
print(df['comments'].head(1))

0    [{'text': 'Going back to school is not identical with giving up. Some founders go back to school and keep working on the startup while there.  However, those do so much worse than the people who work on the startup full-time that going back to school seems, in practice, not too far removed from a death sentence for a startup.Off the top of my head, I'd guess we've had about 8 startups where the founders went back to school.  It doesn't only happen with summer batches.  Founders from winter batches do it too.Usually the reason is that the startup isn't doing very well. However, that judgement depends a lot on how determined the founders are.  One reason we now shy away from funding people still in school is that they often unconsciously want the startup to fail, because the idea of dropping out frightens them.A lot of startups look bad after 3 months.  Someone who's out of school and has to make it work or get a job in a cubicle will say "don't worry, we'll figure out how to make i

In [37]:
df['comments1'] = df['comments'][1] 

ValueError: Length of values (5) does not match length of index (88107)

In [35]:
print(df['text'].head(1))

0    How many summer Y Combinator fundees decided not to continue with their startup and go back to school? and what were the reasons?
Name: text, dtype: object


In [8]:
df1 = pd.read_json('ranking_test.jsonl',  lines = True)
df1

,text,comments
0,"iOS 8.0.1 released, broken on iPhone 6 models,...",[{'text': 'I&#x27;m still waiting for them to ...
1,Ask HN: How do US HNers get their health insur...,[{'text': 'Get it from your employer. It&#x27;...
2,San Diego Researcher Crowdfunding Patent-Free ...,[{'text': 'What I don&#x27;t understand is why...
3,Rethinking the origins of the universe,[{'text': 'I&#x27;m not a physicist. I imagin...
4,SlackTextViewController: A new growing text in...,[{'text': 'As someone that doesn&#x27;t do iOS...
...,...,...
13999,The cat's miaow,"[{'text': 'Meanwhile in the US, Stubbs has bee..."
14000,Facebook’s Piracy Problem,[{'text': 'A radical idea: Maybe our model of ...
14001,Go GC: Solving the Latency Problem in Go 1.5,[{'text': 'Was the presentation more in-depth ...
14002,Understanding Neural Networks Through Deep Vis...,[{'text': 'Ok now I want to &quot;hear&quot; o...
